In [30]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import time
import multiprocess
from multiprocess import Pool

num_procs = multiprocess.cpu_count()
print(num_procs)

16


In [ ]:
df = pd.read_csv('data/series.csv')
airports = pd.read_csv('data/airports.csv')
departures = df[df["departure"].isin(airports["airport"])].copy().drop_duplicates()
arrivals = df[df["arrival"].isin(airports["airport"])].copy().drop_duplicates()
time_ = departures["dep time"].apply(lambda d: datetime.datetime.fromtimestamp(d).time() if not np.isnan(d) else "NaN")
departures["dep time"] = time_
departures["dep time minute"]  = time_.apply(lambda t: np.round(t.hour*60 + t.minute + t.second*0.1)).astype(int)

time_ = arrivals["arr time"].apply(lambda d: datetime.datetime.fromtimestamp(d).time() if not np.isnan(d) else "NaN")
arrivals["arr time"] = time_
arrivals["arr time minute"]  = time_.apply(lambda t: np.round(t.hour*60 + t.minute + t.second*0.1)).astype(int)

In [ ]:
departure_day=departures[departures["week day"]==0]
departure_day

In [19]:
flights_departure= departure_day["callsign"].unique()
flights_arrival= arrivals["callsign"].unique()
flights_departure.shape

(10654,)

In [25]:
def compute(tup):
    fl_dep, dep, time_tolerance, min_occourrency = tup
    regular = 0
    for flight in fl_dep:
        f = dep[dep["callsign"]==flight]["dep time minute"]
        mean, std = f.mean(), f.std()
        if f[(f < mean + time_tolerance) & (f > mean - time_tolerance)].shape[0]/f.shape[0]> min_occourrency:
            regular += 1
    return regular

In [26]:
time_tolerance = 45
min_occourrency = 0.75

len_slice = flights_departure.shape[0]//num_procs
split_fl = [i*len_slice for i in range(num_procs)] + [flights_departure.shape[0]]

split_flights = tuple([(flights_departure[split_fl[i]:split_fl[i+1]], departure_day[departure_day["callsign"].isin(flights_departure[split_fl[i]:split_fl[i+1]])], time_tolerance,min_occourrency) for i in range(num_procs)])

In [27]:
t = time.time()

pool = Pool(num_procs)
reg = sum(pool.map(compute, split_flights))
pool.close()
pool.join()
print(time.time()-t)
print(reg)

3.3402106761932373
9679


In [28]:
from itertools import product
tol = [30,35,40, 45,50,55, 60]
min_oc = [0.70,0.75,0.80,0.85,0.90,0.95]
grid = list(product(tol, min_oc))
    

In [ ]:
fls = []
for point in grid:
    print(point)
    time_tolerance = point[0]
    min_occourrency = point[1]
    split_flights = tuple([(flights_departure[split_fl[i]:split_fl[i+1]], departures[departures["callsign"].isin(flights_departure[split_fl[i]:split_fl[i+1]])], time_tolerance,min_occourrency) for i in range(num_procs)])
    pool = Pool(num_procs)
    fls.append(sum(pool.map(compute, split_flights)))
    pool.close()
    pool.join()

(30, 0.85)
(30, 0.9)
(30, 0.95)
(35, 0.7)
(35, 0.75)
(35, 0.8)
(35, 0.85)
(35, 0.9)
(35, 0.95)
(40, 0.7)
(40, 0.75)
(40, 0.8)
(40, 0.85)
(40, 0.9)
(40, 0.95)
(45, 0.7)
(45, 0.75)
(45, 0.8)
(45, 0.85)
(45, 0.9)
(45, 0.95)
(50, 0.7)
(50, 0.75)


Process ForkPoolWorker-476:
Process ForkPoolWorker-473:
Process ForkPoolWorker-471:
Process ForkPoolWorker-466:
Process ForkPoolWorker-480:
Process ForkPoolWorker-469:
Process ForkPoolWorker-468:
Process ForkPoolWorker-467:
Process ForkPoolWorker-470:
Process ForkPoolWorker-472:
Process ForkPoolWorker-477:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-478:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/andrea/Scrivania/SOSTA/venv/lib/python3.8/site-packages/multiprocess/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):


KeyboardInterrupt: 

Process ForkPoolWorker-465:
Traceback (most recent call last):
  File "/home/andrea/Scrivania/SOSTA/venv/lib/python3.8/site-packages/multiprocess/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
Process ForkPoolWorker-475:


In [ ]:
fls = np.array(fls)
plt.rcParams["figure.figsize"] = (30,25)
plt.rcParams["font.size"] = 25
points = np.array(grid).T
plt.xticks(tol)
plt.yticks(min_oc)
plt.xlabel("TIME TOLERANCE")
plt.xlim(25,65)
plt.ylim(0.65,1)
plt.ylabel("OCCURRENCE TOLERANCE")
plt.title
for i in range(len(grid)):
    plt.annotate(fls[i],(grid[i][0]-1,grid[i][1]-0.001),color='white')
plt.scatter(points[0],points[1], s=fls-5000)
plt.savefig("tol.png")

## Per week day

In [11]:
day =["Monday", "Tuesday", "Wednesday", "Thursday","Friday", "Saturday", "Sunday"]
for j in range(7):
    departures_day =arrivals[arrivals["week day"]==j]
    flights_departure= departures_day["callsign"].unique()
    print(flights_departure.shape)
    len_slice = flights_departure.shape[0]//num_procs
    split_fl = [i*len_slice for i in range(num_procs)] + [flights_departure.shape[0]]
    fls = []
    t = time.time()
    for point in grid:
        partial_time = time.time()
        print(point)
        time_tolerance = point[0]
        min_occourrency = point[1]
        split_flights = tuple([(flights_departure[split_fl[i]:split_fl[i+1]], departures_day[departures_day["callsign"].isin(flights_departure[split_fl[i]:split_fl[i+1]])], time_tolerance,min_occourrency) for i in range(num_procs)])
        pool = Pool(num_procs)
        fls.append(sum(pool.map(compute, split_flights)))
        pool.close()
        pool.join()
        print("paritial", time.time()-partial_time)

    print("time total: ",time.time()-t)

    fls = np.array(fls)
    plt.rcParams["figure.figsize"] = (30,25)
    plt.rcParams["font.size"] = 25
    points = np.array(grid).T
    plt.xticks(tol)
    plt.yticks(min_oc)
    plt.xlabel("TIME TOLERANCE")
    plt.xlim(25,65)
    plt.ylim(0.65,1)
    plt.ylabel("OCCURRENCE TOLERANCE")
    plt.title(day[j])
    for i in range(len(grid)):
        plt.annotate(fls[i],(grid[i][0],grid[i][1]),color='white',horizontalalignment='center',verticalalignment='center')
    plt.scatter(points[0],points[1], s=fls*1.5)
    plt.savefig("plots/arrivals_"+day[j]+".png")
    plt.cla()
    plt.clf()
    plt.close()
    print(j)

(10654,)


NameError: name 'grid' is not defined

In [42]:
arrivals

,icao24,dep time,departure,arr time,arrival,callsign,dep dist,dep alt,arr dist,arr alt,candidate dep airports,candidate arr airports,day,week day,series code,series,arr time minute
0,4cc2a7,1.558355e+09,EBBR,17:11:12,BIKF,ICE67K,935.0,42.0,2909.0,39.0,2,0,2019-05-20,0,EBBR_BIKF_ICE67K _0,0,1032
1,4cc2a2,1.559569e+09,EBBR,18:20:29,BIKF,ICE67K,792.0,2.0,2624.0,69.0,2,0,2019-06-03,0,EBBR_BIKF_ICE67K _0,0,1103
2,4cc27b,1.560169e+09,EBBR,17:01:26,BIKF,ICE67K,1511.0,96.0,4266.0,69.0,2,0,2019-06-10,0,EBBR_BIKF_ICE67K _0,0,1024
3,4cc277,1.560775e+09,EBBR,17:17:20,BIKF,ICE67K,918.0,40.0,2650.0,92.0,2,0,2019-06-17,0,EBBR_BIKF_ICE67K _0,0,1039
4,3412d3,1.561983e+09,EBBR,17:30:15,BIKF,ICE67K,487.0,33.0,3049.0,6.0,2,0,2019-07-01,0,EBBR_BIKF_ICE67K _0,0,1052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1248337,506c21,1.567965e+09,EDDM,20:21:58,LSZH,ADR33D,2210.0,171.0,385.0,50.0,0,7,2019-09-08,6,LSZH_EDDM_ADR33D _6,73151,1227
1248338,506c21,1.568569e+09,EDDM,20:19:59,LSZH,ADR33D,1821.0,49.0,737.0,88.0,0,7,2019-09-15,6,LSZH_EDDM_ADR33D _6,73151,1225
1248339,71c251,1.567952e+09,LOWW,17:17:27,LSZH,KAL9575,1525.0,38.0,1783.0,42.0,0,7,2019-09-08,6,LSZH_LOWW_KAL9575 _6,73152,1040
1248340,71c251,1.568559e+09,LOWW,17:49:26,LSZH,KAL9575,777.0,83.0,1875.0,88.0,0,7,2019-09-15,6,LSZH_LOWW_KAL9575 _6,73152,1072


## Global std

In [47]:
def compute_std(tup):
    fl_dep, dep, tolerance = tup
    std_list = []
    
    for flight in fl_dep:
        f = dep[dep["callsign"]==flight]["dep time minute"]
        mean, std = f.mean(), f.std()
        std_list.append(std)

    return std_list

In [50]:


t = time.time()

pool = Pool(num_procs)
reg = pool.map(compute_std, split_fl)
pool.close()
pool.join()
print(time.time()-t)


9.776390552520752
